# Exploratory Data Analysis For Benin

Importing Libraries

In [ ]:
import matplotlib.pyplot as plt  
import numpy as np
import pandas as pd 
import seaborn as sns
from scipy import stats 
from windrose import WindroseAxes

Loading the Dataset

- The dataset contains 19 variables, 1 object datatype and 18 numerical datatype.

- Total dataset size 525600 rows.

In [ ]:
df=pd.read_csv('/Users/elbethelzewdie/Documents/solar-challenge-week0/solar-challenge-week0/data/benin-malanville.csv')
print(df.info())

In [ ]:
# The first 15 rows from the dataset
df.head(15)

In [ ]:
# The last 5 rows from the dataset
df.tail(5)

Checking for null values in the dataset

- The "Comments" column has 525600 missing values.

In [ ]:
df.isnull().sum()

Drop the "Comments" column

In [ ]:
df = df.drop('Comments', axis=1)

In [ ]:
df.isnull().sum()

Outlier Detection & Basic Cleaning

In [ ]:
df.describe()

In [ ]:
cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

for col in cols:
    plt.figure(figsize=(6,4))
    df.boxplot(column=col)
    plt.title(f"Box Plot of {col}")
    plt.ylabel(col)
    plt.show()

In [ ]:
cols_to_check = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

z_scores = np.abs(stats.zscore(df[cols_to_check], nan_policy='omit'))

# Flag rows where any column has |Z| > 3
outliers = (z_scores > 3).any(axis=1)


print(f"Number of outlier rows: {outliers.sum()}")
df_outliers = df[outliers]
df_outliers.head()

In [ ]:
# Drop rows flagged as outliers
df_clean = df[~outliers].copy()  # keep rows where outliers == False

# Check new shape
print("Original rows:", df.shape[0])
print("Rows after dropping outliers:", df_clean.shape[0])

Time Series Analysis

In [ ]:
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean = df_clean.sort_values('Timestamp')

In [ ]:
unique_count = df_clean['Timestamp'].nunique()
print(f"Number of unique timestamps: {unique_count}")

In [ ]:
start_date = df_clean['Timestamp'].min()
end_date = df_clean['Timestamp'].max()

print(f"Date range: {start_date} to {end_date}")

In [ ]:
df_clean['Month'] = df_clean['Timestamp'].dt.month
df_clean['Hour'] = df_clean['Timestamp'].dt.hour

In [ ]:
# Daily average
daily_avg = df_clean.resample('D', on='Timestamp')[['GHI','DNI','DHI','Tamb']].mean()

# Monthly average
monthly_avg = df_clean.groupby('Month')[['GHI','DNI','DHI','Tamb']].mean()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(daily_avg.index, daily_avg['GHI'], label='GHI')
plt.plot(daily_avg.index, daily_avg['DNI'], label='DNI')
plt.plot(daily_avg.index, daily_avg['DHI'], label='DHI')
plt.plot(daily_avg.index, daily_avg['Tamb'], label='Tamb (°C)')

plt.title('Daily Average Solar Irradiance and Temperature Over Time')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
plt.plot(df_clean['Timestamp'], df_clean['GHI'], label='GHI', alpha=0.8)
plt.plot(df_clean['Timestamp'], df_clean['DNI'], label='DNI', alpha=0.8)
plt.plot(df_clean['Timestamp'], df_clean['DHI'], label='DHI', alpha=0.8)
plt.plot(df_clean['Timestamp'], df_clean['Tamb'], label='Tamb (°C)', alpha=0.8)
plt.title('Solar Irradiance and Temperature over Time')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean['month'] = df_clean['Timestamp'].dt.month
df_clean['hour'] = df_clean['Timestamp'].dt.hour
df_clean['day'] = df_clean['Timestamp'].dt.day

In [ ]:
monthly_avg = df_clean.groupby('month')[['GHI','DNI','DHI','Tamb']].mean()

monthly_avg.plot(kind='bar', figsize=(10,6))
plt.title('Monthly Average Solar and Temperature Patterns')
plt.xlabel('Month')
plt.ylabel('Average Value')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
df_clean['GHI_roll'] = df_clean['GHI'].rolling(window=60).mean()  # e.g., 60-min average

plt.figure(figsize=(14,5))
plt.plot(df_clean['Timestamp'], df_clean['GHI'], label='GHI (Raw)', alpha=0.4)
plt.plot(df_clean['Timestamp'], df_clean['GHI_roll'], label='GHI (Rolling Mean)', color='red')
plt.title('GHI Over Time with Rolling Average')
plt.legend()
plt.show()

In [ ]:
hourly_avg = df_clean.groupby('hour')[['GHI','DNI','DHI','Tamb']].mean()

hourly_avg.plot(kind='bar', figsize=(12,6))
plt.title('Average Solar and Temperature Patterns by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Value')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Cleaning Impact

In [ ]:
mod_clean = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()
print(mod_clean)

In [ ]:
mod_clean.plot(kind='bar', figsize=(6,5), color=['#69b3a2', '#ffcc66'])
plt.title('Average ModA & ModB Readings Before and After Cleaning')
plt.xlabel('Cleaning Flag (0 = Before, 1 = After)')
plt.ylabel('Average Sensor Reading')
plt.xticks(rotation=0)
plt.legend(title='Module')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Correlation & Relationship Analysis

Heatmap 

In [ ]:
corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']

# Compute correlation matrix
corr = df_clean[corr_cols].corr()

# Plot heatmap
plt.figure(figsize=(7,5))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap: Solar & Module Temperature Variables')
plt.show()

Scatter plots

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x='WS', y='GHI', data=df_clean, alpha=0.6)
plt.title('Wind Speed vs GHI')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('GHI')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x='WSgust', y='GHI', data=df_clean, alpha=0.6, color='orange')
plt.title('Wind Gust vs GHI')
plt.xlabel('Wind Gust (m/s)')
plt.ylabel('GHI')
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.scatterplot(x='WD', y='GHI', data=df_clean, alpha=0.6, color='green')
plt.title('Wind Direction vs GHI')
plt.xlabel('Wind Direction (°)')
plt.ylabel('GHI')
plt.show()

Wind & Distribution Analysis

Wind rose

In [ ]:
wind_data = df_clean.dropna(subset=['WS', 'WD'])

plt.figure(figsize=(7,7))
ax = WindroseAxes.from_ax()
ax.bar(wind_data['WD'], wind_data['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_title('Wind Rose: Direction and Speed Distribution', fontsize=12)
ax.set_legend(title='Wind Speed (m/s)')
plt.show()

Histogram

In [ ]:
plt.figure(figsize=(12,5))

# Histogram for GHI
plt.subplot(1,2,1)
sns.histplot(df_clean['GHI'], bins=30, kde=True, color='gold')
plt.title('Distribution of Global Horizontal Irradiance (GHI)')
plt.xlabel('GHI')
plt.ylabel('Frequency')

# Histogram for WS
plt.subplot(1,2,2)
sns.histplot(df_clean['WS'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Wind Speed (WS)')
plt.xlabel('Wind Speed (m/s)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

Temperature Analysis

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.6)
plt.title('Relative Humidity vs Ambient Temperature')
plt.xlabel('RH (%)')
plt.ylabel('Tamb (°C)')
plt.grid(True)
plt.show()

# RH vs GHI
plt.figure(figsize=(8,6))
plt.scatter(df_clean['RH'], df_clean['GHI'], alpha=0.6)
plt.title('Relative Humidity vs GHI')
plt.xlabel('RH (%)')
plt.ylabel('GHI (W/m²)')
plt.grid(True)
plt.show()

# RH vs DNI
plt.figure(figsize=(8,6))
plt.scatter(df_clean['RH'], df_clean['DNI'], alpha=0.6)
plt.title('Relative Humidity vs DNI')
plt.xlabel('RH (%)')
plt.ylabel('DNI (W/m²)')
plt.grid(True)
plt.show()

# RH vs DHI
plt.figure(figsize=(8,6))
plt.scatter(df_clean['RH'], df_clean['DHI'], alpha=0.6)
plt.title('Relative Humidity vs DHI')
plt.xlabel('RH (%)')
plt.ylabel('DHI (W/m²)')
plt.grid(True)
plt.show()

In [ ]:
cols = ['RH', 'Tamb', 'GHI', 'DNI', 'DHI']
corr_matrix = df_clean[cols].corr()
print(corr_matrix)

In [ ]:
df_clean['hour'] = df_clean['Timestamp'].dt.hour
hourly_avg = df_clean.groupby('hour')[['Tamb', 'RH']].mean()

hourly_avg.plot(kind='bar', figsize=(10,5))
plt.title('Average Temperature and Humidity by Hour of Day')
plt.xlabel('Hour')
plt.ylabel('Average Value')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,5))

ax1.plot(df_clean['Timestamp'], df_clean['Tamb'], color='red', label='Temperature (°C)')
ax1.set_xlabel('Timestamp')
ax1.set_ylabel('Temperature (°C)', color='red')

ax2 = ax1.twinx()
ax2.plot(df_clean['Timestamp'], df_clean['RH'], color='blue', label='RH (%)', alpha=0.6)
ax2.set_ylabel('Relative Humidity (%)', color='blue')

plt.title('Temperature and Humidity Over Time')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Bubble Chart

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(
    x=df_clean['GHI'], 
    y=df_clean['Tamb'], 
    s=df_clean['RH'],        # bubble size = RH
    alpha=0.6,
    c=df_clean['RH'],        # color by RH for extra info
    cmap='viridis',
    edgecolors='w'
)

plt.title('GHI vs Temperature (Bubble size = RH)')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Tamb (°C)')
plt.colorbar(label='Relative Humidity (%)')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))

plt.scatter(
    x=df_clean['GHI'], 
    y=df_clean['Tamb'], 
    s=df_clean['BP'],      # scale down BP to reasonable bubble size
    alpha=0.6,
    c=df_clean['BP'],
    cmap='coolwarm',
    edgecolors='w'
)

plt.title('GHI vs Temperature (Bubble size = BP)')
plt.xlabel('GHI (W/m²)')
plt.ylabel('Tamb (°C)')
plt.colorbar(label='Barometric Pressure (hPa)')
plt.grid(True)
plt.show()
